In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/flyhopper/scripts/Julia/Project.toml`


In [2]:
using SymPy

In [11]:
# generate manipulator equation
# TODO: Switch to Symbolics.jl
sympy.init_printing(use_unicode=True)

@syms m0, m1, m2, m3, g, l0, l1, l2, l34
@syms q0()::real t
@syms q1()::real t
@syms q2()::real t
@syms q3()::real t
@syms q0d()::real t
@syms q1d()::real t
@syms q2d()::real t
@syms q3d()::real t
@syms q0dd()::real t
@syms q1dd()::real t
@syms q2dd()::real t
@syms q3dd()::real t

# l34 = l3 + l4  # last link is split into l3 and l4 to mark parallel joint constraint

x0 = l0*sympy.cos(q0(t))
y0 = l0*sympy.sin(q0(t))
x1 = l0*sympy.cos(q0(t)) + l1*sympy.cos(q0(t) + q1(t))
y1 = l0*sympy.sin(q0(t)) + l1*sympy.sin(q0(t) + q1(t))

x2 = l2*sympy.cos(q2(t))
y2 = l2*sympy.sin(q2(t))
x3 = l2*sympy.cos(q2(t)) + l34*sympy.cos(q2(t) + q3(t))
y3 = l2*sympy.sin(q2(t)) + l34*sympy.sin(q2(t) + q3(t))

x0d = diff(x0, t)
y0d = diff(y0, t)
x1d = diff(x1, t)
y1d = diff(y1, t)
x2d = diff(x2, t)
y2d = diff(y2, t)
x3d = diff(x3, t)
y3d = diff(y3, t)

U0 = m0*g*y0 + m1*g*y1
U2 = m2*g*y2 + m3*g*y3

v0_squared = x0d^2 + y0d^2
v1_squared = x1d^2 + y1d^2
v2_squared = x2d^2 + y2d^2
v3_squared = x3d^2 + y3d^2
T0 = sympy.simplify(0.5*m0*v0_squared + 0.5*m1*v1_squared)
T2 = sympy.simplify(0.5*m2*v2_squared + 0.5*m3*v3_squared)

U = U0 + U2
T = T0 + T2

# Le Lagrangian
L = sympy.trigsimp(T - U)
L = L.subs(sympy.Derivative(q0(t), t), q0d(t))  # substitute d/dt q2(t) with q2d
L = L.subs(sympy.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
L = L.subs(sympy.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d
L = L.subs(sympy.Derivative(q3(t), t), q3d(t))  # substitute d/dt q2(t) with q2d

# Lagrange-Euler Equation
LE0 = diff(diff(L, q0d(t)), t) - diff(L, q0(t))
LE1 = diff(diff(L, q1d(t)), t) - diff(L, q1(t))  
LE2 = diff(diff(L, q2d(t)), t) - diff(L, q2(t))
LE3 = diff(diff(L, q3d(t)), t) - diff(L, q3(t))
LE = [LE0, LE1, LE2, LE3]
# LE = sympy.trigsimp(LE)

# subs first derivative
LE = LE.subs(sympy.Derivative(q0(t), t), q0d(t))  # substitute d/dt q1(t) with q1d
LE = LE.subs(sympy.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
LE = LE.subs(sympy.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d
LE = LE.subs(sympy.Derivative(q3(t), t), q3d(t))  # substitute d/dt q1(t) with q1d
# subs second derivative
LE = LE.subs(sympy.Derivative(q0d(t), t), q0dd(t))  # substitute d/dt q1d(t) with q1dd
LE = LE.subs(sympy.Derivative(q1d(t), t), q1dd(t))  # substitute d/dt q1d(t) with q1dd
LE = LE.subs(sympy.Derivative(q2d(t), t), q2dd(t))  # substitute d/dt q2d(t) with q2dd
LE = LE.subs(sympy.Derivative(q3d(t), t), q3dd(t))  # substitute d/dt q1d(t) with q1dd
LE = sympy.expand(sympy.simplify(LE))

# Generalized mass matrix
# M1 = sympy.Poly(LE[1], q1dd(t)).coeffs()
M11 = collect(LE[0], q0dd(t)).coeff(q0dd(t))
M12 = collect(LE[0], q1dd(t)).coeff(q1dd(t))
M13 = collect(LE[0], q2dd(t)).coeff(q2dd(t))
M14 = collect(LE[0], q3dd(t)).coeff(q3dd(t))

M21 = collect(LE[1], q0dd(t)).coeff(q0dd(t))
M22 = collect(LE[1], q1dd(t)).coeff(q1dd(t))
M23 = collect(LE[1], q2dd(t)).coeff(q2dd(t))
M24 = collect(LE[1], q3dd(t)).coeff(q3dd(t))

M31 = collect(LE[2], q0dd(t)).coeff(q0dd(t))
M32 = collect(LE[2], q1dd(t)).coeff(q1dd(t))
M33 = collect(LE[2], q2dd(t)).coeff(q2dd(t))
M34 = collect(LE[2], q3dd(t)).coeff(q3dd(t))

M41 = collect(LE[3], q0dd(t)).coeff(q0dd(t))
M42 = collect(LE[3], q1dd(t)).coeff(q1dd(t))
M43 = collect(LE[3], q2dd(t)).coeff(q2dd(t))
M44 = collect(LE[3], q3dd(t)).coeff(q3dd(t))

M = [M11 M12 M13 M14;
     M21 M22 M23 M24;
     M31 M32 M33 M34;
     M41 M42 M43 M44]

# Gravity Matrix
G = LE
G = G.subs(q0d(t), 0)
G = G.subs(q1d(t), 0)  # must remove q derivative terms manually
G = G.subs(q2d(t), 0)
G = G.subs(q3d(t), 0)
G = G.subs(q0dd(t), 0)
G = G.subs(q1dd(t), 0)
G = G.subs(q2dd(t), 0)
G = G.subs(q3dd(t), 0)

# Coriolis Matrix
# assume anything without qdd minus G is C
C = LE
C = C.subs(q0dd(t), 0)
C = C.subs(q1dd(t), 0)
C = C.subs(q2dd(t), 0)
C = C.subs(q3dd(t), 0)
C = C - G

LE

⎡                                                                             
⎢   g⋅l₀⋅m₀⋅cos(q₀(t)) + g⋅l₀⋅m₁⋅cos(q₀(t)) + g⋅l₁⋅m₁⋅cos(q₀(t) + q₁(t)) + 1.0
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢g⋅l₂⋅m₂⋅cos(q₂(t)) + g⋅l₂⋅m₃⋅cos(q₂(t)) + g⋅l₃₄⋅m₃⋅cos(q₂(t) + q₃(t)) + 1.0⋅l
⎢                                                                             
⎢                                                                             
⎣                                                                            g

   2                2                                                         
⋅l₀ ⋅m₀⋅q0dd(t) + l₀ ⋅m₁⋅q0dd(t) - 2⋅l₀⋅l₁⋅m₁⋅q0d(t

In [12]:
@show M

M = Sym[1.0*l0^2*m0 + l0^2*m1 + 2*l0*l1*m1*cos(q1(t)) + l1^2*m1 l0*l1*m1*cos(q1(t)) + l1^2*m1 0 0; l0*l1*m1*cos(q1(t)) + l1^2*m1 l1^2*m1 0 0; 0 0 1.0*l2^2*m2 + l2^2*m3 + 2*l2*l34*m3*cos(q3(t)) + l34^2*m3 l2*l34*m3*cos(q3(t)) + l34^2*m3; 0 0 l2*l34*m3*cos(q3(t)) + l34^2*m3 l34^2*m3]


4×4 Matrix{Sym}:
 1.0*l0^2*m0 + l0^2*m1 + 2*l0*l1*m1*cos(q1(t)) + l1^2*m1  …                                0
                           l0*l1*m1*cos(q1(t)) + l1^2*m1                                   0
                                                       0     l2*l34*m3*cos(q3(t)) + l34^2*m3
                                                       0                            l34^2*m3

In [13]:
@show C

C = SymMatrix(PyObject Matrix([
[  -2*l0*l1*m1*q0d(t)*q1d(t)*sin(q1(t)) - l0*l1*m1*q1d(t)**2*sin(q1(t))],
[                                         l0*l1*m1*q0d(t)**2*sin(q1(t))],
[-2*l2*l34*m3*q2d(t)*q3d(t)*sin(q3(t)) - l2*l34*m3*q3d(t)**2*sin(q3(t))],
[                                        l2*l34*m3*q2d(t)**2*sin(q3(t))]]))


⎡                                                    2               ⎤
⎢ -2⋅l₀⋅l₁⋅m₁⋅q0d(t)⋅q1d(t)⋅sin(q₁(t)) - l₀⋅l₁⋅m₁⋅q1d (t)⋅sin(q₁(t)) ⎥
⎢                                                                    ⎥
⎢                                2                                   ⎥
⎢                    l₀⋅l₁⋅m₁⋅q0d (t)⋅sin(q₁(t))                     ⎥
⎢                                                                    ⎥
⎢                                                     2              ⎥
⎢-2⋅l₂⋅l₃₄⋅m₃⋅q2d(t)⋅q3d(t)⋅sin(q₃(t)) - l₂⋅l₃₄⋅m₃⋅q3d (t)⋅sin(q₃(t))⎥
⎢                                                                    ⎥
⎢                                 2                                  ⎥
⎣                    l₂⋅l₃₄⋅m₃⋅q2d (t)⋅sin(q₃(t))                    ⎦

In [14]:
@show G

G = SymMatrix(PyObject Matrix([
[ g*l0*m0*cos(q0(t)) + g*l0*m1*cos(q0(t)) + g*l1*m1*cos(q0(t) + q1(t))],
[                                           g*l1*m1*cos(q0(t) + q1(t))],
[g*l2*m2*cos(q2(t)) + g*l2*m3*cos(q2(t)) + g*l34*m3*cos(q2(t) + q3(t))],
[                                          g*l34*m3*cos(q2(t) + q3(t))]]))


⎡g⋅l₀⋅m₀⋅cos(q₀(t)) + g⋅l₀⋅m₁⋅cos(q₀(t)) + g⋅l₁⋅m₁⋅cos(q₀(t) + q₁(t)) ⎤
⎢                                                                     ⎥
⎢                     g⋅l₁⋅m₁⋅cos(q₀(t) + q₁(t))                      ⎥
⎢                                                                     ⎥
⎢g⋅l₂⋅m₂⋅cos(q₂(t)) + g⋅l₂⋅m₃⋅cos(q₂(t)) + g⋅l₃₄⋅m₃⋅cos(q₂(t) + q₃(t))⎥
⎢                                                                     ⎥
⎣                     g⋅l₃₄⋅m₃⋅cos(q₂(t) + q₃(t))                     ⎦